# Reading JSON input file and extracting data

In [ ]:
import json   
import pandas as pd

In [82]:
f1 = open("Corona_02_Split_01.json" , 'r')

lines = f1.readlines()

tweets = {}
tweets['tweets'] = []

for line in lines:
    if line[0] == '{':
        tweets['tweets'].append(json.loads(line)) 

In [100]:
#twitter_list = tweets['tweets']
#twitter_list[0]

# Saving the data into a pandas dataframe

In [84]:
# Load the data into a Pandas DataFrame
twitter_df = pd.DataFrame(twitter_list)

# Display the first 5 rows of the complete twitter DataFrame
twitter_df.head(5)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,favorited,retweeted,filter_level,lang,timestamp_ms,display_text_range,extended_tweet,possibly_sensitive,extended_entities,withheld_in_countries
0,Sun Apr 12 18:27:25 +0000 2020,1249403767180668930,1249403767180668930,RT @nuffsaidny: wishing death on people is wei...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,False,False,low,en,1586716045552,NaN,NaN,NaN,NaN,NaN
1,Sun Apr 12 18:27:25 +0000 2020,1249403768023678982,1249403768023678982,"RT @lale_karanfil: In Turkey, there are 300 th...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,False,False,low,en,1586716045753,NaN,NaN,NaN,NaN,NaN
2,Sun Apr 12 18:27:26 +0000 2020,1249403769193779202,1249403769193779202,RT @umesh_agr: Thank You Sir !!\nReally thankf...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,False,False,low,en,1586716046032,NaN,NaN,NaN,NaN,NaN
3,Sun Apr 12 18:27:26 +0000 2020,1249403769567227906,1249403769567227906,RT @meysimek: Turkey is so stubborn to change ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,False,False,low,en,1586716046121,NaN,NaN,NaN,NaN,NaN
4,Sun Apr 12 18:27:26 +0000 2020,1249403770435493888,1249403770435493888,RT @biannagolodryga: This image.\nThis quote.\...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,False,False,low,en,1586716046328,NaN,NaN,NaN,NaN,NaN


In [85]:
# extract specific columns
twitter_users_df = twitter_data_frame[['created_at', 'id_str','text','user','retweeted_status','quote_count','reply_count','retweet_count','favorite_count','entities']]

In [88]:
twitter_users_df.head(5)

,created_at,id_str,text,user,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,entities
0,Sun Apr 12 18:27:25 +0000 2020,1249403767180668930,RT @nuffsaidny: wishing death on people is wei...,"{'id': 1242817830946508801, 'id_str': '1242817...",{'created_at': 'Sun Apr 12 16:48:01 +0000 2020...,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
1,Sun Apr 12 18:27:25 +0000 2020,1249403768023678982,"RT @lale_karanfil: In Turkey, there are 300 th...","{'id': 1225145123920588805, 'id_str': '1225145...",{'created_at': 'Sun Apr 12 18:02:41 +0000 2020...,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
2,Sun Apr 12 18:27:26 +0000 2020,1249403769193779202,RT @umesh_agr: Thank You Sir !!\nReally thankf...,"{'id': 101007632, 'id_str': '101007632', 'name...",{'created_at': 'Sun Apr 12 13:19:18 +0000 2020...,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
3,Sun Apr 12 18:27:26 +0000 2020,1249403769567227906,RT @meysimek: Turkey is so stubborn to change ...,"{'id': 1230170166614482944, 'id_str': '1230170...",{'created_at': 'Sun Apr 12 18:24:49 +0000 2020...,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
4,Sun Apr 12 18:27:26 +0000 2020,1249403770435493888,RT @biannagolodryga: This image.\nThis quote.\...,"{'id': 4707764075, 'id_str': '4707764075', 'na...",{'created_at': 'Sun Apr 12 12:40:06 +0000 2020...,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ..."


# Connection to PostgreSQL Database

In [92]:
import psycopg2 # psycopg2 is a PostgreSQL database adapter for the Python , and it provides a Python interface for connecting to and interacting with PostgreSQL databases.

In [99]:

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="TwitterDatabase",
    user="postgres",
    password="India@2194"
)


### Creating a cursor object to execute PostgreSQL commands

In [101]:
# Create a cursor object to execute PostgreSQL commands
cursor = conn.cursor()

# Inserting input data into the relational database